# Main class - `EEGForwardModel.m`

In [ ]:
%%file EEGForwardModel.m
classdef EEGForwardModel < EEGSignalGenerator
    % By methods in this class we obtain signals in the space of the electrodes, i.e.
    % the sources project on electrodes by multiplying the matrix of signals
    % by the leadfield matrix. This way one gets signals on the electrodes,
    % which already contains noise from the background and interference sources.
    % The only noise that does not come from this operation is the measurement noise.
    %
    % EEGForwardModel Properties:
    %    SETUP - setup for simulation,
    %    MODEL - model variables.
    %
    % EEGForwardModel Methods:
    %    setleadfields - calculating leadfields.
    
    properties(GetAccess='public', SetAccess='public')
        % Properties of the class
    end

    methods
        function obj = EEGForwardModel()
            % Constructor of the EEGForwardModel class
        end
        function obj = setleadfields(obj)
            % Set leadfields.

            obj.MODEL = geometryrandomsampling(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = geometryindices(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = geometrycoordinates(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = geometrydeepsources(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = geometryperturbation(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = geometryleadfieldscomputation(obj.SETUP, obj.MODEL, obj.MATS);
            obj.MODEL = forwardmodeling(obj.SETUP, obj.MODEL);
        end
        function obj = setpreparations(obj)
            % Creates signal from source activities, background and interference noise by
            % utilizing signal to noise ratio.

            obj.MODEL = preparesnrsadjustment(obj.SETUP, obj.MODEL);
            obj.MODEL = prepareleadfieldconfiguration(obj.SETUP, obj.MODEL);
            obj.MODEL = preparemeasuredsignals(obj.SETUP, obj.MODEL);
            obj.MODEL = store2eeglab(obj.SETUP, obj.MODEL, obj.MATS);
        end        
    end
end

# Auxiliary functions

In [ ]:
%%file geometryrandomsampling.m
function model = geometryrandomsampling(SETUP, MODEL, MATS)
    % Draws random ROIs and vertices.

    model = MODEL;
    model.sim_geo_cort.numROIs = size(MATS.sel_atl.Atlas(MATS.sel_atl.atl).Scouts, 2);

    disp('CC: using random ROI locations');
    if SETUP.SEED
        rng(SETUP.SEEDS(5));
    end
    model.sim_geo_cort.lstROIs = randsample([1:model.sim_geo_cort.numROIs], size(SETUP.SRCS, 1));
    
    % Overwrite random ROIs with predefined if pROIs list is not empty
    if length( SETUP.pROI ) ~= 0
        disp('CC: using (some) predefined ROI locations');
        for ii = 1:length( SETUP.pROI )
            % overwrite as many items as possible given the predefined list
            model.sim_geo_cort.lstROIs(ii) = SETUP.pROI(ii);
            % truncate the final list to the size equal to the number of sources
            model.sim_geo_cort.lstROIs = model.sim_geo_cort.lstROIs(1:size(SETUP.SRCS, 1));
        end
    end

    if SETUP.rPNT
        disp('CC: using random source locations (TODO)');
    else
        disp('CC: using predefined source locations (TODO)');
    end
end

In [ ]:
%%file geometryindices.m
function model = geometryindices(SETUP, MODEL, MATS)
    % Finds indices corresponding to drawn ROIs.

    model = MODEL;
    model.sim_geo_cort.indROIs = [];
    for ii = 1:length(model.sim_geo_cort.lstROIs)
        tmp_roi = model.sim_geo_cort.lstROIs(ii);
        model.sim_geo_cort.indROIs(ii).pntNum00 = MATS.sel_atl.Atlas(MATS.sel_atl.atl).Scouts(tmp_roi).Vertices';
        model.sim_geo_cort.indROIs(ii).triNum00 = find(all(ismember(MATS.sel_atl.tri, model.sim_geo_cort.indROIs(ii).pntNum00), 2));
    end
end

In [ ]:
%%file geometrycoordinates.m
function model = geometrycoordinates(SETUP, MODEL, MATS)
   % Finds coordinates corresponding to drawn ROIs.

    model = MODEL;
    model.sim_geo_cort.distrROIs = SETUP.SRCS;
    model.sim_geo_cort.distrROIsSum = sum(SETUP.SRCS, 2);
    model.sim_geo_cort.bulkSRC = {};
    if SETUP.SEED
        rng(SETUP.SEEDS(5));
    end
    for ii = 1:length(model.sim_geo_cort.distrROIsSum)
        model.sim_geo_cort.bulkSRC{ii, 1} = randsample(model.sim_geo_cort.indROIs(ii).pntNum00, model.sim_geo_cort.distrROIsSum(ii));
    end
    model.sim_geo_cort.splitSRC = {};
    for ii = 1:length(model.sim_geo_cort.distrROIsSum)
        model.sim_geo_cort.splitSRC(ii, :) = mat2cell(model.sim_geo_cort.bulkSRC{ii}, model.sim_geo_cort.distrROIs(ii, :), 1)';
    end

    if SETUP.DEBUG
       model.sim_geo_cort.splitSRC(1, 1);
       model.sim_geo_cort.splitSRC{1, 1};
    end

    if SETUP.rPNT
        disp('CC: using random source locations');
    else
        disp('CC: using predefined source locations');
        %model.sim_geo_cort.splitSRC{1, 1} = [5441; 5506; 5481; 5283; 5144; 5063; 5823; 5987; 6154; 6065; 6166; 6346; 6087; 6446; 6367; 6726; 6859; 6829; 6613; 6698; 6966; 7162; 7227; 6949; 6996; 7365; 7247; 7480];
        model.sim_geo_cort.splitSRC{1, 1} = model.sim_geo_cort.splitSRC{1, 1}(1:SETUP.SRCS(1, 1));
        if SETUP.DEBUG
           model.sim_geo_cort.splitSRC{1, 1};
        end
    end

    model.sim_geo_cort.mergeSRC = {};
    for ii = 1:3
        model.sim_geo_cort.mergeSRC{ii} = cat(1, model.sim_geo_cort.splitSRC{:, ii});
    end
    model.sim_geo_cort.pos_orig = {};
    model.sim_geo_cort.ori_orig = {};
    model.sim_geo_cort.pos_pert = {};
    model.sim_geo_cort.ori_pert = {};
    for ii = 1:3
        model.sim_geo_cort.pos_orig{ii} = MATS.sel_atl.pnt(model.sim_geo_cort.mergeSRC{ii}, :);
        model.sim_geo_cort.ori_orig{ii} = MATS.sel_atl.vn1(model.sim_geo_cort.mergeSRC{ii}, :);
        model.sim_geo_cort.pos_pert{ii} = model.sim_geo_cort.pos_orig{ii};
        model.sim_geo_cort.ori_pert{ii} = model.sim_geo_cort.ori_orig{ii};
    end
    
    
    model.sim_geo_cort.ori_orig_arch = model.sim_geo_cort.ori_orig;
    for ii = 1:3
        V = model.sim_geo_cort.ori_orig{ii};
        M = vecnorm(V,2,2);
        model.sim_geo_cort.ori_orig_unit{ii} = bsxfun(@rdivide, V, M);
    end
    
    for ii = 1:3
        model.sim_geo_cort.ori_orig_AzEl{ii} = EEGStaticMethods().rawCartToSph(model.sim_geo_cort.ori_orig_unit{ii})
    end
    
    model.sim_geo_cort.ori_todo_AzEL = {SETUP.AzElSrcS,SETUP.AzElIntS,SETUP.AzElBcgS}
    
    for ii = 1:3
        model.sim_geo_cort.ori_done_AzEl{ii} = model.sim_geo_cort.ori_orig_AzEl{ii} + model.sim_geo_cort.ori_todo_AzEL{ii}
    end
    
    for ii = 1:3
        model.sim_geo_cort.ori_done_OXYZ{ii} = EEGStaticMethods().rawSphToCart(model.sim_geo_cort.ori_done_AzEl{ii})
    end
    
    for ii = 1:3
        model.sim_geo_cort.ori_orig{ii} = model.sim_geo_cort.ori_done_OXYZ{ii}
        model.sim_geo_cort.ori_pert{ii} = model.sim_geo_cort.ori_done_OXYZ{ii}
    end

    
end

In [ ]:
%%file geometrydeepsources.m
function model = geometrydeepsources(SETUP, MODEL, MATS)
   % Finds coordinates corresponding to drawn deep sources (thalami).

    model = MODEL;
    if SETUP.thalamus == "ico"
        model.sim_geo_deep = MATS.sel_geo_deep_icosahedron642;
    else
        model.sim_geo_deep = MATS.sel_geo_deep_thalami;
    end

    if SETUP.SEED
        rng(SETUP.SEEDS(5));
    end
    model.sim_geo_deep.bulkSRC = randsample(1:size(model.sim_geo_deep.pnt, 1), sum(SETUP.DEEP))';

    [model.sim_geo_deep.mergeSRC{1}, model.sim_geo_deep.mergeSRC{2}, model.sim_geo_deep.mergeSRC{3}] = deal([]);

    model.sim_geo_deep.mergeSRC{1, 1} = model.sim_geo_deep.bulkSRC(1:SETUP.DEEP(1));
    model.sim_geo_deep.mergeSRC{1, 2} = model.sim_geo_deep.bulkSRC(1 + SETUP.DEEP(1):SETUP.DEEP(1) + SETUP.DEEP(2));
    model.sim_geo_deep.mergeSRC{1, 3} = model.sim_geo_deep.bulkSRC(1 + SETUP.DEEP(1) + SETUP.DEEP(2):SETUP.DEEP(1) + SETUP.DEEP(2) + SETUP.DEEP(3));

    model.sim_geo_deep.pos_orig = {};
    model.sim_geo_deep.ori_orig = {};
    model.sim_geo_deep.pos_pert = {};
    model.sim_geo_deep.ori_pert = {};

    for ii = 1:3
        model.sim_geo_deep.pos_orig{ii} = model.sim_geo_deep.pnt(model.sim_geo_deep.mergeSRC{ii}, :);
        model.sim_geo_deep.ori_orig{ii} = model.sim_geo_deep.vn1(model.sim_geo_deep.mergeSRC{ii}, :);
        model.sim_geo_deep.pos_pert{ii} = model.sim_geo_deep.pos_orig{ii};
        model.sim_geo_deep.ori_pert{ii} = model.sim_geo_deep.ori_orig{ii};
    end

    model.sim_geo = model.sim_geo_cort;
    for ii = 1:3
        model.sim_geo.pos_orig{ii} = [model.sim_geo.pos_orig{ii}; model.sim_geo_deep.pos_orig{ii}];
        model.sim_geo.ori_orig{ii} = [model.sim_geo.ori_orig{ii}; model.sim_geo_deep.ori_orig{ii}];
        model.sim_geo.pos_pert{ii} = [model.sim_geo.pos_pert{ii}; model.sim_geo_deep.pos_pert{ii}];
        model.sim_geo.ori_pert{ii} = [model.sim_geo.ori_pert{ii}; model.sim_geo_deep.ori_pert{ii}];
    end;
    
    
    
    
    
    
    
end

In [ ]:
%%file geometryperturbation.m
function model = geometryperturbation(SETUP, MODEL, MATS)
    % Perturb original vertices coordinates by shifting randomly
    % within a cube o a given side length centred at the original
    % source location as well as orientation of each source within
    % prescribed azimuth and elevation, both centred at the original
    % source orientation.

    % WARNING:: suppress warning about MATLAB version and CFG tracking 
    warning(SETUP.WARNINGS, 'all');

    providepathstonecessarytoolboxes();
    model = MODEL;
    
    if SETUP.SEED
        rng(SETUP.SEEDS(5));
    end
    for ii = 1:3  
        model.sim_geo.ori_pert{ii} = normr(EEGStaticMethods().rawSphToCart(EEGStaticMethods().rawCartToSph(model.sim_geo.ori_orig{ii}) + [SETUP.CONE * 0.5 * (2*rand(size(model.sim_geo.ori_pert{ii}, 1), 2) - 1), zeros(size(model.sim_geo.ori_pert{ii}, 1), 1)]));
    end
    tmp_count = 0;
    for ii = 1:3
        for jj = 1:size(model.sim_geo.pos_pert{ii}, 1)
            tmp_srcInside = false;
            while ~tmp_srcInside
                tmp_count = tmp_count + 1;
                model.sim_geo.pos_pert{ii}(jj, :) = model.sim_geo.pos_orig{ii}(jj, :) + SETUP.CUBE * 0.5 * (2*rand([1, 3]) - 1);
                tmp_srcInside = bounding_mesh(model.sim_geo.pos_pert{ii}(jj, :), MATS.sel_msh.bnd(1).pnt, MATS.sel_msh.bnd(1).tri);
            end
        end
    end
    if SETUP.TELL
        disp(['CYBERCRAFT:: Perturbation took ', tmp_count, ' iterations']);
    end
end

In [ ]:
%%file geometryleadfieldscomputation.m
function model = geometryleadfieldscomputation(SETUP, MODEL, MATS)
    % Generates lead-fields using FieldTrip.

    model = MODEL;
    
    % WARNING:: suppress warning about MATLAB version and CFG tracking 
    warning(SETUP.WARNINGS, 'all');

    tmp_cfg            = [];
    tmp_cfg.grid.unit  = 'mm';
    tmp_cfg.reducerank = 'no';
    tmp_cfg.normalize  = 'no';
    tmp_cfg.elec       = MATS.sel_ele;
    tmp_cfg.vol        = MATS.sel_vol;

    ii = 1;
    tmp_cfg.grid.pos          = model.sim_geo.pos_orig{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.ori_orig{ii});
    model.sim_lfg_SrcActiv_orig.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_SrcActiv_orig.LFG = cat(2, model.sim_lfg_SrcActiv_orig.lfg.leadfield{:});

    tmp_cfg.grid.pos          = model.sim_geo.pos_pert{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.ori_pert{ii});
    model.sim_lfg_SrcActiv_pert.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_SrcActiv_pert.LFG = cat(2, model.sim_lfg_SrcActiv_pert.lfg.leadfield{:});

    ii = 2;
    tmp_cfg.grid.pos          = model.sim_geo.pos_orig{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.pos_orig{ii});
    model.sim_lfg_IntActiv_orig.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_IntActiv_orig.LFG = cat(2, model.sim_lfg_IntActiv_orig.lfg.leadfield{:});

    tmp_cfg.grid.pos          = model.sim_geo.pos_pert{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.pos_pert{ii});
    model.sim_lfg_IntActiv_pert.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_IntActiv_pert.LFG = cat(2, model.sim_lfg_IntActiv_pert.lfg.leadfield{:});

    ii = 3;
    tmp_cfg.grid.pos          = model.sim_geo.pos_orig{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.pos_orig{ii});
    model.sim_lfg_BcgActiv_orig.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_BcgActiv_orig.LFG = cat(2, model.sim_lfg_BcgActiv_orig.lfg.leadfield{:});

    tmp_cfg.grid.pos          = model.sim_geo.pos_pert{ii};
    tmp_cfg.grid.mom          = transpose(model.sim_geo.pos_pert{ii});
    model.sim_lfg_BcgActiv_pert.lfg = ft_prepare_leadfield(tmp_cfg);
    model.sim_lfg_BcgActiv_pert.LFG = cat(2, model.sim_lfg_BcgActiv_pert.lfg.leadfield{:});
    
    %csvwrite('leadfield.csv', model.sim_lfg_SrcActiv_orig.LFG);
end

In [ ]:
%%file forwardmodeling.m
function model = forwardmodeling(SETUP, MODEL)
    % Forward modeling (lead-field multiplicated by signals in source space).

    model = MODEL;
    
    for kk = 1:SETUP.K00
        model.sim_sig_SrcActiv.sigSNS_pre(:, :, kk) = (model.sim_lfg_SrcActiv_orig.LFG * model.sim_sig_SrcActiv.sigSRC_pre(:, :, kk)')';
        model.sim_sig_SrcActiv.sigSNS_pst(:, :, kk) = (model.sim_lfg_SrcActiv_orig.LFG * model.sim_sig_SrcActiv.sigSRC_pst(:, :, kk)')';
        model.sim_sig_IntActiv.sigSNS_pre(:, :, kk) = (model.sim_lfg_IntActiv_orig.LFG * model.sim_sig_IntActiv.sigSRC_pre(:, :, kk)')';
        model.sim_sig_IntActiv.sigSNS_pst(:, :, kk) = (model.sim_lfg_IntActiv_orig.LFG * model.sim_sig_IntActiv.sigSRC_pst(:, :, kk)')';
        model.sim_sig_BcgActiv.sigSNS_pre(:, :, kk) = (model.sim_lfg_BcgActiv_orig.LFG * model.sim_sig_BcgActiv.sigSRC_pre(:, :, kk)')';
        model.sim_sig_BcgActiv.sigSNS_pst(:, :, kk) = (model.sim_lfg_BcgActiv_orig.LFG * model.sim_sig_BcgActiv.sigSRC_pst(:, :, kk)')';
    end
end

In [ ]:
%%file preparesnrsadjustment.m
function model = preparesnrsadjustment(SETUP, MODEL)
    % Adjusts SNR ratio for all source signals.
    % Power of the signals of interest is kept constant while noise
    % signals powers are adjusted according to SETUP.S*NR.

    model = MODEL;
    
    model.sim_sig_AdjSNRs.SrcActivPre = model.sim_sig_SrcActiv.sigSNS_pre;
    model.sim_sig_AdjSNRs.SrcActivPst = model.sim_sig_SrcActiv.sigSNS_pst;
    model.sim_sig_AdjSNRs.IntActivPre = model.sim_sig_IntActiv.sigSNS_pre;
    model.sim_sig_AdjSNRs.IntActivPst = model.sim_sig_IntActiv.sigSNS_pst;
    model.sim_sig_AdjSNRs.BcgActivPre = model.sim_sig_BcgActiv.sigSNS_pre;
    model.sim_sig_AdjSNRs.BcgActivPst = model.sim_sig_BcgActiv.sigSNS_pst;
    model.sim_sig_AdjSNRs.MesNoisePre = model.sim_sig_MesNoise.sigSNS_pre;
    model.sim_sig_AdjSNRs.MesNoisePst = model.sim_sig_MesNoise.sigSNS_pst;
    for kk = 1:SETUP.K00
        model.sim_sig_AdjSNRs.IntActivPre(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPre(:,:,kk), model.sim_sig_AdjSNRs.IntActivPre(:,:,kk), SETUP.SINR);
        model.sim_sig_AdjSNRs.IntActivPst(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPst(:,:,kk), model.sim_sig_AdjSNRs.IntActivPst(:,:,kk), SETUP.SINR);
        model.sim_sig_AdjSNRs.BcgActivPre(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPre(:,:,kk), model.sim_sig_AdjSNRs.BcgActivPre(:,:,kk), SETUP.SBNR);
        model.sim_sig_AdjSNRs.BcgActivPst(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPst(:,:,kk), model.sim_sig_AdjSNRs.BcgActivPst(:,:,kk), SETUP.SBNR);
        model.sim_sig_AdjSNRs.MesNoisePre(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPre(:,:,kk), model.sim_sig_AdjSNRs.MesNoisePre(:,:,kk), SETUP.SMNR);
        model.sim_sig_AdjSNRs.MesNoisePst(:,:,kk) = EEGStaticMethods().rawAdjTotSNRdB(model.sim_sig_AdjSNRs.SrcActivPst(:,:,kk), model.sim_sig_AdjSNRs.MesNoisePst(:,:,kk), SETUP.SMNR);
    end
end

In [ ]:
%%file prepareleadfieldconfiguration.m
function model = prepareleadfieldconfiguration(SETUP, MODEL)
    % Lead-field configuration according to perturbation settings
    % and reducing rank of lead-field matrix of interfering sources.

    model = MODEL;
    % Note: in this and functions subsections we aim at following closely notation from the paper.

    % H_Src taken from FieldTrip
    if SETUP.H_Src_pert
        model.H_Src = model.sim_lfg_SrcActiv_pert.LFG;
    else
        model.H_Src = model.sim_lfg_SrcActiv_orig.LFG;
    end

    % H_Int taken from FieldTrip
    if SETUP.H_Int_pert
        model.H_Int = model.sim_lfg_IntActiv_pert.LFG;
    else
        model.H_Int = model.sim_lfg_IntActiv_orig.LFG;
    end

    % reduced-rank lead-field for patch constraints
    [model.U_model.H_Int model.Si_model.H_Int model.V_model.H_Int] = svd(model.H_Int);

    for ii = SETUP.IntLfgRANK + 1:size(model.H_Int, 2)
        model.Si_model.H_Int(ii, ii) = 0;
    end

    model.H_Int = model.U_model.H_Int * model.Si_model.H_Int * model.V_model.H_Int';
end

In [ ]:
%%file preparemeasuredsignals.m
function model = preparemeasuredsignals(SETUP, MODEL)
    % Preparing mesured signals for pre and post intervals.

    model = MODEL;

    model.y_Pre =     SETUP.SigPre*model.sim_sig_AdjSNRs.SrcActivPre ...
        + SETUP.IntPre*model.sim_sig_AdjSNRs.IntActivPre ...
        + SETUP.BcgPre*model.sim_sig_AdjSNRs.BcgActivPre ...
        + SETUP.MesPre*model.sim_sig_AdjSNRs.MesNoisePre;

    model.y_Pst =     SETUP.SigPst*model.sim_sig_AdjSNRs.SrcActivPst ...
        + SETUP.IntPst*model.sim_sig_AdjSNRs.IntActivPst ...
        + SETUP.BcgPst*model.sim_sig_AdjSNRs.BcgActivPst ...
        + SETUP.MesPst*model.sim_sig_AdjSNRs.MesNoisePst;
        
    %csvwrite('input.csv', model.y_Pst);
end

In [ ]:
%%file store2eeglab.m
function model = store2eeglab(SETUP, MODEL, MATS)
    % Embed supFunSim simulation data in EEGLAB data structure.

    model = MODEL;
    tmp_EEG.setname     = '';
    tmp_EEG.filename    = '';
    tmp_EEG.filepath    = '';
    tmp_EEG.subject     = '';
    tmp_EEG.group       = '';
    tmp_EEG.condition   = '';
    tmp_EEG.session     = [];
    tmp_EEG.comments    = '';
    tmp_EEG.nbchan      = 0;
    tmp_EEG.trials      = 0;
    tmp_EEG.pnts        = 0;
    tmp_EEG.srate       = 1;
    tmp_EEG.xmin        = 0;
    tmp_EEG.xmax        = 0;
    tmp_EEG.times       = [];
    tmp_EEG.data        = [];
    tmp_EEG.icaact      = [];
    tmp_EEG.icawinv     = [];
    tmp_EEG.icasphere   = [];
    tmp_EEG.icaweights  = [];
    tmp_EEG.icachansind = [];
    tmp_EEG.chanlocs    = [];
    tmp_EEG.urchanlocs  = [];
    tmp_EEG.chaninfo    = [];
    tmp_EEG.ref         = [];
    tmp_EEG.event       = [];
    tmp_EEG.urevent     = [];
    tmp_EEG.eventdescription = {};
    tmp_EEG.epoch       = [];
    tmp_EEG.epochdescription = {};
    tmp_EEG.reject      = [];
    tmp_EEG.stats       = [];
    tmp_EEG.specdata    = [];
    tmp_EEG.specicaact  = [];
    tmp_EEG.splinefile  = '';
    tmp_EEG.icasplinefile = '';
    tmp_EEG.dipfit      = [];
    tmp_EEG.history     = '';
    tmp_EEG.saved       = 'no';
    tmp_EEG.etc         = [];
    
    tmp_sig_pre = permute(MODEL.y_Pre, [2,1,3]);
    tmp_sig_pst = permute(MODEL.y_Pst, [2,1,3]);
    tmp_sig_all = cat(2, tmp_sig_pre, tmp_sig_pst);

    tmp_EEG.data        = tmp_sig_all;
    tmp_EEG.nbchan      = size(tmp_sig_all, 1);
    tmp_EEG.trials      = size(tmp_sig_all, 3);
    tmp_EEG.pnts        = size(tmp_sig_all, 2);
    tmp_EEG.srate       = SETUP.SRATE;

    tmp_EEG.xmin        = (-size(tmp_sig_pre, 2) - 1)/tmp_EEG.srate;
    tmp_EEG.xmax        = size(tmp_sig_pst, 2)/tmp_EEG.srate;
    tmp_EEG.times       = linspace(tmp_EEG.xmin*1000, tmp_EEG.xmax*1000, tmp_EEG.pnts);

    tmp_EEG.chanlocs = struct('labels', cellstr(char(MATS.sel_ele.label{:})));
    tmp_X = num2cell(MATS.sel_ele.elecpos(:, 1));
    tmp_Y = num2cell(MATS.sel_ele.elecpos(:, 2));
    tmp_Z = num2cell(MATS.sel_ele.elecpos(:, 3));
    [tmp_EEG.chanlocs.X] = tmp_X{:};
    [tmp_EEG.chanlocs.Y] = tmp_Y{:};
    [tmp_EEG.chanlocs.Z] = tmp_Z{:};    
    
    tmp_EEG = pop_chanedit( tmp_EEG, 'convert', 'cart2topo' )

    [tmp_EEG.chanlocs(:).type] = deal('EEG');
    [tmp_EEG.chanlocs(:).ref]  = deal('average');

    tmp_EEG.ref         = 'averef'; % 'averef' | 'common'
    
    model.EEG = tmp_EEG;
end